In [2]:
""" Essai partie 3 "automatisée" par moi en récupérant des trucs d'un peu partout
Etapes principales:
1- check_FP_output_loc (ou truc appelé différemment) pour regarder dans quelle(s) zone(s) géo se trouvent les résultats FP
<!> SI intéressant quand spec001_mr > max(spec001_mr)/2 --> <!!> PAS spec001_mr, c'est aggrégation de spec001_mr sur height et time par ex
    2- trouver les zones d'intérets (spec001_mr > max(spec001_mr)/2) dans le FP output
    3- récup les données GLM, ABI etc. qui correspondent à ces zones et moments d'intérêts
    4- regarde si correspondance entre FP output et les données satellite + réseau au sol etc.
    <!> SI correspondance entre FP et données éclairs
    5- récup les informations d'intérêt (infos chimiques et électriques)
    6- (plus tard) flag les vols et tout pour la db IAGOS (mais on verra plus tard pour ça)

-?- Remarques:
- est-ce qu'on garderait pas juste les infos intéressantes (max rt) dans un nouv nc au lieu de faire argmax etc. tout le temps dans chq nouveau nb ?
    
En cours en ce moment:
- refonte de check_FP_output
""";

In [ ]:
"""
Check_FP_output goal: find which geographical area is covered by max residence time in FP output to determine which satallite data etc. should be used
NON 1- Get the extreme lon and lat for each geographical area and CONVERT IT to FP grid
NON 2- (???) Add extra column to FP output nc with geographical area indicator (like GEO_REGIONS in soft-io ?) for max res time values 
1- Open netcdf
2- Recup argmax lon-lat
3- regarder dans quelle zone géo ça se trouve
    --> how ? regarder les min et les max des lat et des lon ?
        --> si chevauche deux zones on récup les données des deux zones du coup j'imagine ?
    --> connaître les zones de couverture des satellites
    --> OBJ = savoir quelles données de satellites aller récupérer 
        --> doit savoir quel satellite couvre quelle zone
        --> trouver un moyen pas trop dégueu de déterminer quelles zones couvertes par les temps de res max FP
    
PS: pour ce qui est du spec001_mr > max/2 est-ce que j'en ai besoin ou pas?
    --> si utilise ici peut permettre de réduire la zone géographique où on veut récup des données satellites (mais est-ce que utile?)
    --> si utilise ailleurs --> comme aggregated over time and height, how can I use it to OOOOH maybe using argmax ?
        --> add time to list of dim over which finding argmax ?? or maybe just put time and not lon lat ? JSP 
""";

In [1]:
# imports
from pathlib import Path
import xarray as xr
import fpout
import xarray_extras as xrx

In [2]:
def check_str_to_Path(path_to_check):
    """
    Function to check if path_to_check is Path instance. 
    If not, check if path_to_check is a str and convert it to pathlib.Path object
    :path_to_check: should be Path or str
    :return: path_to_check as a pathlib.Path object or raise exception if path_to_check not Path nor str
    """
    if not isinstance(path_to_check, Path):
        if isinstance(path_to_check, str):
            path_to_check = Path(path_to_check)
        else:
            raise TypeError(f'given path should be str or Path object, not {type(path_to_check)}')
    return path_to_check

In [3]:
def find_FP_output_flight_dirs(flexpart_output_path):
    """
    Function returning a list of directories containing FLEXPART outputs for each flight analysed
    :flexpart_output_path: PosixPath or str, directory containing FP outputs
    :return: sorted list of directory paths containing the FP results for each flight (one directory per flight analysed)  
    :rtype: <list>[ <PosixPath>, ... ]
    """
    # check if flexpart_output_path is instance of Path class
    flexpart_output_path = check_str_to_Path(flexpart_output_path)
    # return list of directories in flexpart_output_path
    return sorted(flexpart_output_path.glob("*/"))

In [11]:
def find_FP_output_netcdf_files(FP_out_flight_dir, nc_output_dir_name='10j_100k_output'):
    """
    Function to find path of netcdf files in a FP output result directory
    :FP_out_flight_dir: PosixPath, path to FP output directory for the flight
    :nc_output_dir_name: str, name of subdirectory in which netcdf file is supposed to be stored
    :return: list of netcdf files in the FP output subdirectory (usually only one nc file per flight)
    :rtype: <list>[ <PosixPath>, ... ]
    """
    FP_out_flight_dir = check_str_to_Path(FP_out_flight_dir)
    dir_path = FP_out_flight_dir / nc_output_dir_name
    # if nc_output_dir_name is in the FP output directory, we look for netcdf file
    if dir_path in sorted(FP_out_flight_dir.iterdir()):
        return sorted(dir_path.glob("*.nc"))
    else:
        raise FileNotFoundError(f'Directory {dir_path} does not exist')

#### <!> might not work if nested directories for FP output --> see 2015 directory structure in /o3p/macc flexpart output

In [7]:
def find_all_FP_output_netcdf_files(FP_out_flights_dirs):
    """
    Function to get all FP output netcdf files
    :FP_out_flights_dirs: PosixPath or str, base directory contanining FP output directories
    :return: list of all netcdf files paths
    :rtype: <list>[ <PosixPath>, ... ]
    """
    FP_out_nc_files = []
    FP_out_flights_dirs = check_str_to_Path(FP_out_flights_dirs)
    for FP_out_dir in FP_out_flights_dirs:
        FP_out_nc_files += find_FP_output_netcdf_files(FP_out_dir)
    return sorted(FP_out_nc_files)

In [ ]:
def get_spec001_mr_argmax_dataArray(FP_out_nc_path):
    FP_out_nc_path = check_str_to_Path(FP_out_nc_path) # convert potential str path to PosixPath
    # check that we got a netcdf file
    if not FP_out_nc_path.endswith(".nc"):
        raise ValueError(f'Invalid file {FP_out_nc_path}, expecting netcdf file (path ending with .nc)')
    else:
        ds = fpout.open_fp_dataset(FP_out_nc_path) # use open_dataset in the future (when dev finished)
        da = ds.spec001_mr
    

---

In [4]:
FP_out_dirs = find_FP_output_flight_dirs('/o3p/patj/SOFT-IO-LI/flexpart10.4/flexpart_v10.4_3d7eebf/src/exercises/soft-io-li/')
len(FP_out_dirs)

28

In [6]:
find_FP_output_netcdf_files(FP_out_dirs[0])

[PosixPath('/o3p/patj/SOFT-IO-LI/flexpart10.4/flexpart_v10.4_3d7eebf/src/exercises/soft-io-li/flight_2018_001_1h_05deg/10j_100k_output/grid_time_20180603150000.nc')]

In [8]:
FP_out_nc_files = find_all_FP_output_netcdf_files(FP_out_dirs)
len(FP_out_nc_files)

28

In [9]:
FP_out_nc_files[0]

PosixPath('/o3p/patj/SOFT-IO-LI/flexpart10.4/flexpart_v10.4_3d7eebf/src/exercises/soft-io-li/flight_2018_001_1h_05deg/10j_100k_output/grid_time_20180603150000.nc')